## Julia installation

Julia can be downloaded from [Julia](https://julialang.org/downloads/) and is automatically recognized by JupyterLab if it has already been installed.  Julia can also be called by simlink.

## Adding packages on Jupyter notebook

After launching a Jupyter notebook for Julia, you first need to add a package manager `Pkg` and add any necessary packages via `Pkg`.

In [ ]:
using Pkg #install a package manager
Pkg.add(url="https://github.com/cumc/SuSiEGLMM.jl.git") # download from the source

#other necessary packages
Pkg.add(["Statistics", "Distributions", "StatsBase", "Random", "LinearAlgebra", "DelimitedFiles", "Distributed", "GLM"])
Pkg.add("Plots")

## Distributed computing

Parallelization (`@distributed`) is performed at the chromosome level; that is, a set of SNPs in each chromosome is assigned to each worker (or process).  One can generate workers up to the number of chromosomes. After that packages need to be loaded with `@everyhwere`, so that all workers can access the packages.  Note that this distributed computing does not have to send all data to all workers; data are accessible on the main process only.


In [ ]:
using Distributed 
addprocs(2) # for this small data set case

using Statistics, Distributions, StatsBase, Random, LinearAlgebra
@everywhere using SuSiEGLMM

In [ ]:
procs() # can check workers' id's. 
#can also use `nprocs()`, `nworkers()` to check the number of processes, workers, resepctively

## Reading and preprocessing data

We need `genotypes`, `traits(or phenotypes)`, and `SNP information (SNP names, chromosomes, positions)`, or  `covariates(optional)`.  Genotypes, phenotypes, and covariates need to be imputed for missing values and their types are either Matrix{Float64} or Vector{Float64} (double precision).  One can check their types using the function, `typeof()`.    Since Julia is type-specific, if they are not double precision, use `convert` to change their types.
For instance, on the REPL

```julia
Julia> typeof(pheno)
Vector{Any} (alias for Array{Any, 1})

Julia> pheno = convert(Vector{Float64},pheno)

```

In [ ]:
using DelimitedFiles #reading files

In [2]:
pwd() #check the working directory

"/Users/jeankim/GIT/SuSiEGLMM.jl/vignettes"

In [ ]:
# use a relative path to read a genotype file
# the first row (column names) is skipped 
@time geno=readdlm(string(@__DIR__,"/../testdata/causal3/fam_folder/ascertained_fam_genotype_12_10.txt";skipstart=1); 

Check for the detail on [readdlm](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/)

In [ ]:
# read SNP info using an abosolute path
info=readdlm(homedir()*"/GIT/SuSiEGLMM.jl/testdata/causal3/fam_folder/ascertained_fam_12_10.bim");

In [ ]:
# check the data
info[1:5,:]

* Genotypes should be imputed (e.g. filling out by the SNP mean) to be normalized.
* **Note!!** Julia is column-majored; looping by column is fast and efficient.

In [ ]:
# check if there are missing values (or "NA") and impute them 
X = geno[:,6:end]

for j =axes(X,2)
    idx = findall(X[:,j].=="NA")
    X[idx,j].= missing
    X[idx,j] .= mean(skipmissing(X[:,j]))
end

# normalization by SNP 
X1= (X.-mean(X,dims=2))./std(X,dims=2) # column-wise standardization

- read a phenotype file 
- `header= true` separates a matrix of phenotypes (pheno[1]) from their column names (pheno[2])

```julia
julia> pheno=readdlm(homedir()*"/GIT/SuSiEGLMM.jl/testdata/causal3/fam_folder/ascertained_fam_phenotype_12_10.txt";header=true);

#covariates (optional) : sex
julia> C = pheno[1][:,end-1]
julia> C[C.==1].=0.0
julia> C[C.==2].=1.0

#phenotypes (not needed for simulation)
julia> y=pheno[1][:,end]
julia> y= convert(Vector{Float64},y); 

```

## Simulation

We need a particular type of arrays for SNP information for distributed computing: type `struct`: `GenoInfo`.  Fine mapping for SuSiE-GLM is run by `fineQTL_glm`. For help, type `?GenoInfo`, `?fineQTL_glm` about the detail, respectively.

In [ ]:
# a struct of arrays for SNP information: `SNP names, chromosomes, positions` in this order
G= GenoInfo(info[:,2],info[:,1],info[:,3])

In [ ]:
# assign workers to seeds
Seed(124)

In [ ]:
n,p = size(X1) # get the dimenson of the genotype matrix
L=3; B=100;q=5;

b_true=zeros(p);
b_1s=zeros(B);b_2s=zeros(B); b_3s=zeros(B);

res=[];Tm=zeros(B);

In [ ]:
# simulations for 3 causal variants, 2 chromosome cases

for j = 1:B
    # give 3 signals only to the first chromosome
    b_true[1]= randn(1)[1] 
    b_true[2]=randn(1)[1]
    b_true[3]=randn(1)[1]
    b_1s[j] = b_true[1]
    b_2s[j] = b_true[2]
    b_3s[j] = b_true[3]
    # generating X directly from N(0,1)
    # X= randn(n,p)
    # generating covariates and the effects
    X₀ = randn(n,q)
    δ=randn(q) 
    #generating binary outcome
    Y= logistic.(X1*b_true+X₀*δ).>rand(n) 
    Y=convert(Vector{Float64},Y)
    
  t0=@elapsed  res0= fineQTL_glm(G,Y,X1,X₀;L=L,tol=1e-4) # can adjust the tolerance
    #  t0=@elapsed  res0= fineQTL_glm(G,Y,X1;L=L) for no covariates (intercept only)
    res=[res;res0]; Tm[j]=t0
end


In [ ]:
println("min, median, max runtimes for susie-glm are $(minimum(Tm)), $(median(Tm)),$(maximum(Tm)).")

In [ ]:
# Posterior estimates
b̂1=zeros(B);b̂2=zeros(B);b̂3=zeros(B);
for j=1:B
A = sum(res[2j-1].α.*res[2j-1].ν,dims=2)[:,1]    
b̂1[j]=A[1]
b̂2[j]=A[2]
b̂3[j]=A[3]
end

#PIPs
α̂1 = [maximum(res[2j-1].α[1,:]) for j=1:B]
α̂2=[maximum(res[2j-1].α[2,:]) for j=1:B]
α̂3=[maximum(res[2j-1].α[3,:]) for j=1:B]

In [ ]:
using Plots
ll=@layout[a;b]; 

p1=scatter(b_1s,b̂1,xlabel= "True effects", ylabel="Posterior estimate",label=false,title="SuSiE-GLM+random covariates")
p2=scatter(b_1s,α̂1, xlabel="True effects",ylabel="PIP",label=false)
plot(p1,p2,layout=ll)

In [ ]:
# 2x2 subplots
l2=@layout[a b;c d]

p3=scatter(b_2s,b̂2,xlabel= "True effects", ylabel="Posterior estimate",label=false,title="SuSiE-GLM+random covariates")
p4=scatter(b_2s,α̂2, xlabel="True effects",ylabel="PIP",label=false)
p5=scatter(b_3s,b̂3,xlabel= "True effects", ylabel="Posterior estimate",label=false)
p6=scatter(b_3s,α̂3, xlabel="True effects",ylabel="PIP",label=false)

plot(p3,p5,p4,p6,layout=ll2)

### FYI, Julia from R/Python 
----

- Well-maintained packages: 
    - [JuliaCall](https://github.com/Non-Contradiction/JuliaCall) among others (`XRJulia`, `RJulia` ) in R
    - [PyJulia](https://github.com/JuliaPy/pyjulia) in Python: can also use in Julia REPL after `using PyCall`

- Documentations: 
   - JuliaCall vignette : https://nbviewer.org/github/Non-Contradiction/JuliaCall/blob/master/example/JuliaCall_in_Jupyter_R_Notebook.ipynb
   - PyJulia documentation: https://pyjulia.readthedocs.io/en/latest/
